In [ ]:
import os
import glob
import torch
print(torch.__version__)
from sklearn.metrics import  f1_score

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
          transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
image_data_root = '/content/drive/MyDrive/Colab_Notebooks/fake_audio/image_dataset/'
image_datasets = {x: datasets.ImageFolder(os.path.join(image_data_root, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

2.1.0+cu121


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class_names = image_datasets['train'].classes
print('class names:',class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:',device)

class names: ['fake', 'real']
device: cuda:0


In [ ]:

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  torch.save(model.state_dict(),'best_model_params.pt')
  best_acc = 0.0
  best_f1=0.0
  train_loss_list=[]
  val_loss_list=[]
  train_f1_list=[]
  val_f1_list=[]
  for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0
      batch_f1=0
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        if phase == 'train':
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        batch_f1+=f1_score(y_true=labels.detach().cpu().numpy(),y_pred=preds.detach().cpu().numpy()) * inputs.size(0)
      #add f1
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]
      epoch_f1 = batch_f1 / dataset_sizes[phase]
      if phase=='train':
        train_loss_list.append(epoch_loss)
        train_f1_list.append(epoch_f1)
      else:
        val_loss_list.append(epoch_loss)
        val_f1_list.append(epoch_f1)

      print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {epoch_f1}')
      if phase=='val' and epoch_f1>best_f1:
        best_f1 = epoch_f1
      if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          torch.save(model.state_dict(), 'best_model_params.pt')
    print(f'Best val Acc: {best_acc:4f}')
    model.load_state_dict(torch.load('best_model_params.pt'))
  return model,train_loss_list,train_f1_list,val_loss_list,val_f1_list

In [ ]:
model_ft = models.resnet34(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001, betas=(0.9, 0.999))

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft,train_loss_list,train_f1_list,val_loss_list,val_f1_list= train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=13)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 56.3MB/s]


Epoch 0/12
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


train Loss: 0.5001 Acc: 0.7884 F1: 0.7890008512624298


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

val Loss: 0.0754 Acc: 0.9744 F1: 0.9743760115268385
Best val Acc: 0.974933
Epoch 12/12
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


train Loss: 0.0313 Acc: 0.9886 F1: 0.9886363735794803


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


val Loss: 0.0682 Acc: 0.9744 F1: 0.9746181737285347
Best val Acc: 0.974933


In [ ]:
y_true=[]
y_pred=[]
model_ft.eval()

for inputs, labels in dataloaders['val']:
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(False):
          outputs = model_ft(inputs)
          _, preds = torch.max(outputs, 1)
      label_predicted = preds.detach().cpu().numpy().tolist()
      label_real = labels.detach().cpu().numpy().tolist()
      y_true.extend(label_real)
      y_pred.extend(label_predicted)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
f1_score(y_true=y_true,y_pred=y_pred)

0.9750927397986222